# Exercise 1: Static Page Web crawling
Implement a web crawler that retrieves the (estimated) gender of a list of given first names
from the website
https://www.gpeters.com/names/baby-names.php
The source file `names.csv` provides a list of names.

In [14]:
import pandas as pd
names_df = pd.read_csv("names.csv")
names_df["gender_result"] = None
names_df["likelyhood_result"] = None
names_df["popularity_result"] = None
names_df["match"] = None

In [15]:
import requests
from bs4 import BeautifulSoup
my_url = "https://www.gpeters.com/names/baby-names.php"
my_header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0"}

In [16]:
def lookup_name(row:pd.Series)->pd.Series:
    request_url = f"{my_url}?name={row['name']}"
    content = requests.get(url=request_url,headers=my_header).content
    soup = BeautifulSoup(content,"html.parser")

    row["gender_result"] = "female" if "girl" in soup.find("div",{"class": "gender-result"}).find("h1").contents[0] else "male"
    row["match"] = row["gender_result"] == row["sex_whattoexpectcom"]

    number_of_results=len(soup.find("div",{"class": "gender-result"}).find_all("p"))
    try:
        if number_of_results == 1:
            row["likelyhood_result"] = None
            row["popularity_result"] = float(soup.find("div",{"class": "gender-result"}).find_all("p")[0].contents[0].split(" ")[5])
        elif number_of_results == 2:
            row["likelyhood_result"] = float(soup.find("div",{"class": "gender-result"}).find_all("p")[0].contents[0].split(" ")[6])
            row["popularity_result"] = float(soup.find("div",{"class": "gender-result"}).find_all("p")[1].contents[0].split(" ")[5])
    except Exception as e:
        row["gender_result"] = "error"
    return(row)

In [18]:
#names_df = names_df.apply(lookup_name, axis=1)
names_df = names_df.head(10).apply(lookup_name, axis=1)

In [20]:
names_df.to_csv("checked_names.csv",index=False)